In [60]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import concurrent.futures
import csv
import json
import urllib
import uuid

import numpy
import renderapi
import shapely.geometry
import shapely.ops
import shapely.affinity
import pandas as pd

In [62]:
ra_stack = "20201020_ra_trial0_correctedmoving"
montage_stack = "20201020_alignment_montages"

ra_pm_collection = "v1dd_409828_L1_onscope_issues_corrected_rough"

dataset = "v1dd"
project = "409828_L1"

ds_scale = 0.01

In [95]:
ra_stack = "20210305_redo1_ra"
montage_stack = "20201203_rmt_alignment_montages"

ra_pm_collection = [
    "H19_06_356_2_2_1_rough_matches_redo1",
    "H19_06_356_2_2_1_rough_matches_redo",
    "H19_06_356_2_2_1_rough_matches"
]

dataset = "human"
project = "H19_06_356_2_2_1"

ds_scale = 0.01

In [251]:
ra_stack = "full_affine_with_anchors_zs7924_ze27882_lam_1e5_registered"
montage_stack = "em_2d_montage_solved_py"

ra_pm_collection = "chunk_rough_align_point_matches"

dataset = "Minnie"
project = "17797_1R"

ds_scale = 0.01

In [219]:
ra_stack = "20210428_rmt_tps25_depth4_redoneaff"
montage_stack = "20210326_alignment_montages"

ra_pm_collection = "21617_R1_rough_20210426_downsampled_mapped"

dataset = "Inhibitory"
project = "21617_R1"

ds_scale = 0.01

In [252]:
render_connect = {
   "owner": "TEM",
    "project": "17797_1R",
    "port": 8888,
    "host": "em-131db2.corp.alleninstitute.org",
    "client_scripts": ""
}
render_connect['project'] = project
r = renderapi.connect(**render_connect)

In [253]:
def combine_resolvedtiles(rts_l):
    tforms = {i.transformId: i for l in (rts.transforms for rts in rts_l) for i in l}
    tspecs = list({ts.tileId: ts for l in (rts.tilespecs for rts in rts_l) for ts in l}.values())
    
    return renderapi.resolvedtiles.ResolvedTiles(transformList=tforms, tilespecs=tspecs)

In [254]:
def angle_between_points(pts_a, pts_b, observer_pts=None):
    """get angle between points in NxM arrays with optional observer points
    where N is number of points and M is dimension of points
    """
    if observer_pts is None:
        observer_pts = numpy.zeros_like(pts_a)
    a = pts_a - observer_pts
    b = pts_b - observer_pts
    return numpy.arccos(
        numpy.einsum("ij,ij->i", a, b) /  # vectorized dot product
        numpy.linalg.norm(a, axis=1) /
        numpy.linalg.norm(b, axis=1))


In [255]:
ra_zs = renderapi.stack.get_z_values_for_stack(ra_stack, render=r) 

In [256]:
num_threads = 60
with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as e:
    rtfut_to_z = {e.submit(renderapi.resolvedtiles.get_resolved_tiles_from_z, ra_stack, z, render=r): z for z in ra_zs}
    
    z_to_rts = {rtfut_to_z[fut]: fut.result() for fut in concurrent.futures.as_completed(rtfut_to_z)}
    

In [257]:
matchgroups = {ts.layout.sectionId for ts in (i for l in (rts.tilespecs for rts in z_to_rts.values()) for i in l)}

with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as e:
    futs = [e.submit(renderapi.pointmatch.get_matches_with_group, ra_pm_collection, g, render=r) for g in matchgroups]
    
    allmatches = [i for l in (fut.result() for fut in concurrent.futures.as_completed(futs)) for i in l]

In [258]:
tileId_to_tileId_match = {(m["pId"], m["qId"]): m for m in allmatches}

In [259]:
combined_resolvedtiles = combine_resolvedtiles(z_to_rts.values())
tileId_to_tilespecs = {ts.tileId: ts for ts in combined_resolvedtiles.tilespecs}

In [260]:
def match_worldcoords_ts(ts, matcharr, sharedTransforms=[]):
    # p_matches = numpy.array(match["matches"]["p"]).T
    # q_matches = numpy.array(match["matches"]["q"]).T
    
    return renderapi.transform.utils.estimate_dstpts(
        ts.tforms, src=matcharr, reference_tforms=sharedTransforms)
    # q_tformed = renderapi.transform.utils.estimate_dstpts(
    #     q_ts.tforms, src=q_matches, reference_tforms=sharedTransforms)
    # return q_tformed - p_tformed

# TODO include weights?
def residuals_from_ts(p_ts, q_ts, match, sharedTransforms=[]):
    p_matches = numpy.array(match["matches"]["p"]).T
    q_matches = numpy.array(match["matches"]["q"]).T
    
    p_tformed = renderapi.transform.utils.estimate_dstpts(
        p_ts.tforms, src=p_matches, reference_tforms=sharedTransforms)
    q_tformed = renderapi.transform.utils.estimate_dstpts(
        q_ts.tforms, src=q_matches, reference_tforms=sharedTransforms)
    return q_tformed - p_tformed


In [261]:
tileId_to_tileId_tformed_pts = {}
for (pId, qId), match in tileId_to_tileId_match.items():
    try:
        p_ts = tileId_to_tilespecs[pId]
        q_ts = tileId_to_tilespecs[qId]
    except KeyError:
        continue
    p_matches = numpy.array(match["matches"]["p"]).T
    q_matches = numpy.array(match["matches"]["q"]).T
    

    tileId_to_tileId_tformed_pts[(pId, qId)] = (
        match_worldcoords_ts(
            p_ts, p_matches, combined_resolvedtiles.transforms),
        match_worldcoords_ts(
            q_ts, q_matches, combined_resolvedtiles.transforms)
    )


In [262]:
# center of mass calculation

# for now, proxying by doing center of source point matches
tileId_to_tileId_residuals = {}
tileId_to_tileId_angular_residuals = {}

for (pId, qId), (p_pts, q_pts) in tileId_to_tileId_tformed_pts.items():
    p_ctr = p_pts.mean(axis=0)

    ang_residuals = angle_between_points(p_pts, q_pts, p_ctr)
    residuals = p_pts - q_pts
    
    tileId_to_tileId_residuals[(pId, qId)] = residuals
    tileId_to_tileId_angular_residuals[(pId, qId)] = ang_residuals

<ipython-input-254-ff4aca974ccd>:9: RuntimeWarning: invalid value encountered in arccos
  return numpy.arccos(


In [263]:
# angular residuals
import numpy as np
median_angular_residuals = []
for k in tileId_to_tileId_angular_residuals.items():
    median_angular_residuals.append(np.nanmedian(k[1]))

In [264]:
mard = []
for k in tileId_to_tileId_angular_residuals.items():
    mard.append(np.nanmedian(np.rad2deg(k[1])))

In [271]:
df = pd.DataFrame(data={"median residual": median_angular_residuals})
df.to_pickle("/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/paper/{}_rough_angular_residuals_radians.pkl".format(dataset))

In [272]:
df = pd.DataFrame(data={"median residual": mard})
df.to_pickle("/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/paper/{}_rough_angular_residuals_degrees.pkl".format(dataset))

In [270]:
for m in median_angular_residuals:
    if m > 5:
        print(m)

In [ ]:
tileId_to_z = {ts.tileId: ts.z for ts in combined_resolvedtiles.tilespecs}

In [ ]:
def get_ctrthreshold_filter(arr, threshold):
    return numpy.linalg.norm(arr - arr.mean(axis=0), axis=1) > threshold

In [ ]:
depth = 2
bad_pIds, bad_qIds = zip(*[(pId, qId) for (pId, qId), angarr in tileId_to_tileId_angular_residuals.items()
    if abs(tileId_to_z[pId] - tileId_to_z[qId]) < depth
    and numpy.rad2deg(angarr[get_ctrthreshold_filter(
        tileId_to_tileId_tformed_pts[(pId, qId)][0], 35)].max()) >= 5])